In [43]:
from pdfminer.high_level import extract_text
import os
import pandas as pd

#pdf_path = "./pdf_folder/IN479454(B)_NonRelavant.pdf"
pdf_path = "./pdf_folder/IN202311060784(A)_Relavant.pdf"
file_name = os.path.basename(pdf_path)  # Get just the file name
full_text = extract_text(pdf_path)

# Optional: replace line breaks with spaces to normalize
full_text = full_text.replace("\n", " ")


✅ 2) Fuzzy matching the start of claims

In [5]:
# import re

# # Look for common variations of the claims section heading
# #match = re.search(r'(I\s*/\s*We\s+Claim\s*:.*)', full_text, re.IGNORECASE | re.DOTALL)
# #match = re.search(r'(I\s*/\s*We\s+Claim\s*:.*)', full_text, re.IGNORECASE | re.DOTALL)
# match = re.search(
#                 r'('
#                 r'(?:I\s*/\s*)?We\s+Claim\s*:|'            # I/We Claim:
#                 r'I\s+Claim\s*:|'                         # I Claim:
#                 r'We\s+Claim\s*:|'                        # We Claim:
#                 r'Claims\s*:|'                            # Claims:
#                 r'What\s+(?:is\s+)?claimed\s+is\s*:|'     # What is claimed is:
#                 r'What\s+we\s+claim\s+is\s*:'
#                 r')\s*(.*)',                              # capture everything after the heading
#                 full_text,
#                 re.IGNORECASE | re.DOTALL
#             )
# if match:
#     claims_text = match.group(2).strip()
# else:
#     print("⚠️ Claims section not found.")
#     claims_text = None

# # Assuming claims_text is your variable with extra spaces
# claims_text = re.sub(r'\s+', ' ', claims_text).strip()
# claims_text

In [54]:
import re

# Step 1: Extract the full claims section using flexible headers
match = re.search(
    r'('
    r'(?:I\s*/\s*)?We\s+Claim\s*:|'            # I/We Claim:
    r'I\s+Claim\s*:|'                         # I Claim:
    r'We\s+Claim\s*:|'                        # We Claim:
    r'Claims\s*:|'                            # Claims:
    r'What\s+(?:is\s+)?claimed\s+is\s*:|'     # What is claimed is:
    r'What\s+we\s+claim\s+is\s*:'
    r')\s*(.*)',                              # Capture everything after the heading
    full_text,
    re.IGNORECASE | re.DOTALL
)

if match:
    # Clean and extract the claims section
    claims_text = re.sub(r'\s+', ' ', match.group(2)).strip()

    # Step 2: Extract only the first claim (from "1." to "2.")
    claim_match = re.search(r'1\.\s(.*?)(?=\s*\d+\.\s)', claims_text)
    if claim_match:
        first_claim = "1. " + claim_match.group(1).strip()
    else:
        print("⚠️ Could not isolate claim 1.")
        first_claim = None
else:
    print("⚠️ Claims section not found.")
    first_claim = None

# Output
first_claim
claims_text=first_claim
claims_text


'1. A method for managing vehicle to vehicle communication, comprising: providing a mobile application in a computing device present with a user; providing permissions to the mobile application; detecting fluctuation in motion a vehicle using a gyroscope; activating crash detection if major fluctuations are detected; measuring decrease in speed of the vehicle if the minor detection is detected.'

✅ 3) Summarize claims using a free LLM

In [51]:
# from transformers import pipeline

# if claims_text:
#     summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
#     summary = summarizer(claims_text[:512], max_length=512, min_length=20, do_sample=False)
#     # print("\n📝 Summary of Claims:\n")
#     # print(summary[0]['summary_text'])
#     claims_text_fb=summary[0]['summary_text']

#     summarizer = pipeline("summarization", model="google/pegasus-cnn_dailymail")
#     summary = summarizer(claims_text[:512], max_length=512, min_length=20, do_sample=False)
#     # print("\n📝 Summary of Claims:\n")
#     # print(summary[0]['summary_text'])

#     claims_text_google=summary[0]['summary_text']
#     claims_text_google=claims_text_google.replace('<n>', '')

#     print(claims_text_fb)
#     print(claims_text_google)


# # Create DataFrame with two columns: filename and text
# df = pd.DataFrame({
#     'file_name': [file_name],
#     'claims_text_fb': [claims_text_fb],
#     'claims_text_google': [claims_text_google]
# })

In [53]:
# df

In [47]:
################################################################### LLM summarization########################

In [48]:
#Using llama3
import requests

prompt = "Summarize this text in plain language in a single paragraph:\n\n" + claims_text

response = requests.post(
    "http://localhost:11434/api/generate",
    json={"model": "llama3", "prompt": prompt, "stream": False}
    # ,timeout=120
)
result = response.json()
print("\n📝 Summary from llama3:\n")
print(result.get("response", "[No response]"))


📝 Summary from llama3:

Here's a summary of the text in plain language:

This method helps manage communication between vehicles by using a smartphone app and sensors. The app is installed on a user's phone, which has permission to access certain features. The phone uses its gyroscope (a sensor that measures motion) to detect when a vehicle is moving or slowing down. If the app detects major changes in motion, it triggers a crash detection alert. If it detects smaller changes, such as a decrease in speed, it can measure and record this data too.


In [25]:
# #Using gemma3
# import requests

# prompt = "Summarize this text in plain language in a single paragraph:\n\n" + cleaned_claims_text

# response = requests.post(
#     "http://localhost:11434/api/generate",
#     json={"model": "gemma3", "prompt": prompt, "stream": False},
#     timeout=120
# )
# result = response.json()
# print("\n📝 Summary from gemma3:\n")
# print(result.get("response", "[No response]"))


In [26]:
# #Using Misral
# import requests

# prompt = "Summarize this text in plain language in a single paragraph:\n\n" + claims_text

# response = requests.post(
#     "http://localhost:11434/api/generate",
#     json={"model": "mistral", "prompt": prompt, "stream": False},
#     timeout=120
# )
# result = response.json()
# print("\n📝 Summary from mistral:\n")
# print(result.get("response", "[No response]"))

In [49]:
from transformers import pipeline

classifier = pipeline("text-classification", model="./bert-claims-fg", tokenizer="./bert-claims-fg")

#Relavant
#sample_text = "The patent claims describe a vehicle management system and method that uses a server to communicate with a target vehicle and multiple other vehicles. Each vehicle has at least one sensor that detects the combustion state of fuel in the engine. The server stores a trained model related to engine control, which uses sensor data and gas station data (including location and fuel supply time period, but not fuel composition) to calculate a correction amount for the combustion state of the fuel. This correction amount is then transmitted to the target vehicle, which adjusts its fuel combustion accordingly. The trained model may also consider data related to the traveling position of the vehicles. The sensor data may include information about the engine's internal pressure, rotational speed, exhaust gas temperature, filter pressure difference, and nitrogen oxide concentration. The system also includes a vehicle capable of communicating with the server and adjusting its fuel combustion based on the received correction amount. The method involves calculating the correction amount for a target vehicle and correcting its fuel combustion state accordingly."
sample_text=result.get("response", "[No response]")
classifier(sample_text)

[{'label': 'LABEL_1', 'score': 0.5845538377761841}]

In [50]:
from transformers import pipeline

classifier = pipeline("text-classification", model="./bert-claims-fg2", tokenizer="./bert-claims-fg2")

#Relavant
#sample_text = "The patent claims describe a vehicle management system and method that uses a server to communicate with a target vehicle and multiple other vehicles. Each vehicle has at least one sensor that detects the combustion state of fuel in the engine. The server stores a trained model related to engine control, which uses sensor data and gas station data (including location and fuel supply time period, but not fuel composition) to calculate a correction amount for the combustion state of the fuel. This correction amount is then transmitted to the target vehicle, which adjusts its fuel combustion accordingly. The trained model may also consider data related to the traveling position of the vehicles. The sensor data may include information about the engine's internal pressure, rotational speed, exhaust gas temperature, filter pressure difference, and nitrogen oxide concentration. The system also includes a vehicle capable of communicating with the server and adjusting its fuel combustion based on the received correction amount. The method involves calculating the correction amount for a target vehicle and correcting its fuel combustion state accordingly."
#sample_text=claims_text[:512]
sample_text=result.get("response", "[No response]")
classifier(sample_text)

[{'label': 'LABEL_1', 'score': 0.7748509645462036}]

In [50]:
sample_text

'1.  A method for managing vehicle to vehicle communication, comprising:   providing a mobile application in a computing device present with   a user;   providing permissions to the mobile application;   detecting fluctuation in motion a vehicle using a gyroscope;   activating crash detection if major fluctuations are detected;   measuring decrease in speed of the vehicle if the minor detection is   detected. 2.  The method as claimed in claim 1, wherein if the major fluctuation is   detected, the method fur'